In [ ]:
!pip install keras

In [ ]:
import keras
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')


In [ ]:
import pickle
from pathlib import Path

with open(f"{Path.home()}/.keras/datasets/cifar-100-python/meta", mode='rb') as meta_file: 
    label_names = pickle.load(meta_file, encoding='bytes')

# Load image labels
coarse_labels = [label.decode('utf-8') for label in label_names[b'coarse_label_names']]
fine_labels = [label.decode('utf-8') for label in label_names[b'fine_label_names']]

In [ ]:
# Let's see how example image looks like
%matplotlib inline
import matplotlib.pyplot as plt
image_index = 400  # Change this number to get another image
plt.imshow(x_train[image_index])
print(fine_labels[y_train[image_index][0]])

In [ ]:
image_matrix = x_train[image_index].astype('float32')
image_matrix /= 255
print(image_matrix)

In [ ]:
# Example of programmatic access to prediction instance
import json

import numpy as np
import requests

prediction_instance_url = None # Enter URL from nctl predict launch here
auth_header = None  # Enter authorization header content (Bearer ...) from nctl predict launch here

if not prediction_instance_url or not auth_header:
    raise ValueError('Fill prediction_instance_url and auth_header values.')

# Append method verb to prediction instance url
prediction_instance_url += ':predict'
    
# Body contains image which we want to predict
body = {"instances": [image_matrix.tolist()]}
result = requests.post(prediction_instance_url, headers={'Authorization': auth_header}, json=body, verify=False)
print(result.text)

# Get predicted class
prediction_values = json.loads(result.text)['predictions'][0]
predicted_class_index = np.argmax(prediction_values)
predicted_class = fine_labels[predicted_class_index]
print(f'Predicted class: {predicted_class}')